In [9]:
### Day Ahead Price
# Following https://freedium.cfd/https://medium.com/datons/entso-e-api-with-python-energy-analysis-in-europe-4943d4121a15
# pip install entsoe-py
import importlib
import entsoe
importlib.reload(entsoe)

import pandas as pd

client = entsoe.EntsoePandasClient(api_key="9122fb71-9b84-493a-bb9b-052bc2275cf8")

start = pd.Timestamp('20210101', tz='UTC')
end = pd.Timestamp('20240401', tz='UTC')

df = client.query_day_ahead_prices(
    country_code='DE_LU',
    start=start, end=end
)
df.to_csv("germany-day-ahead-prices.csv")

In [12]:
df.head(10)

2021-01-01 01:00:00+01:00    48.19
2021-01-01 02:00:00+01:00    44.68
2021-01-01 03:00:00+01:00    42.92
2021-01-01 04:00:00+01:00    40.39
2021-01-01 05:00:00+01:00    40.20
2021-01-01 06:00:00+01:00    39.63
2021-01-01 07:00:00+01:00    40.09
2021-01-01 08:00:00+01:00    41.27
2021-01-01 09:00:00+01:00    44.88
2021-01-01 10:00:00+01:00    45.00
Freq: 60min, dtype: float64

In [13]:
import requests as r
import pandas as pd
from datetime import datetime, timedelta


class EpexSpot():

    def __init__(self):
        self.url = 'https://api.awattar.at/v1/marketdata?'

    def get_spot_prices(self, start, end):

        start_timestamp = int(start.timestamp())
        end_timestamp = int(end.timestamp())

        if start_timestamp > end_timestamp:
            raise Exception("Start must be smaller than end!")

        response = r.get(url= self.url + f'start={start_timestamp}000&end={end_timestamp}000')
        response_df = pd.DataFrame(response.json()['data'])
        response_df['start_date'] = pd.to_datetime(response_df.start_timestamp, unit='ms')


        return response_df[['start_date', 'marketprice', 'unit']]


e = EpexSpot()
prices = e.get_spot_prices(datetime(2021, 1, 1), datetime(2024, 4, 1))
df.to_csv("germany-intraday-prices.csv")  # TODO Confirm this is intraday price